In [1]:
### Testing to understand why some LRP blocks are blank
### If same signs for a row in last layer, this happens, have to depend on a good random seed
### Or fix bias at some high value in the last layer

In [2]:
### Import the required libraries
import numpy as np
import scipy
import matplotlib.pyplot as plt
import cmocean

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")

import tensorflow as tf
import tensorflow.keras as keras
from keras import metrics
from keras.constraints import NonNeg
import innvestigate
from sklearn.model_selection import train_test_split

import os
from os.path import join
import sys

import xarray as xr
import xmitgcm
from xmitgcm import open_mdsdataset
import ecco_v4_py as ecco

import random

# See if GPUs are available
from keras import backend as K
if bool(K._get_available_gpus()):
    print("Running on GPU")
else:
    print("Running on CPU")

# Append to sys.path the absolute path to src/XAIRT
path_list = os.path.abspath('').split('/')
path_src_XAIRT = ''
for link in path_list[:-1]:
    path_src_XAIRT = path_src_XAIRT+link+'/'
sys.path.append(path_src_XAIRT+'/src')

# Now import module XAIRT
from XAIRT import *

### https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed ###
### https://keras.io/examples/keras_recipes/reproducibility_recipes/ ###
SEED = 42
keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

tf.compat.v1.disable_eager_execution()

Running on GPU


2024-04-17 21:24:39.983869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 21:24:41.427858: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38221 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:21:00.0, compute capability: 8.0
2024-04-17 21:24:41.430217: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38221 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0
2024-04-17 21:24:41.432629: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/rep

In [3]:
### Sverdrup 
# mainDir_r4  = '/scratch2/pillarh/eccov4r4'
# mainDir_r5  = '/scratch2/pillarh/eccov4r5'
# gridDir  = mainDir_r4 + '/GRID'
# thetaDir = mainDir_r5 + '/V4r5/diags_daily/SST_day_mean'
# thetaDir_ext = mainDir_r5 + '/V4r5_ext_2020_2023_Jun/diags_daily/SST_day_mean'

## LS6
mainDir_r4 = '/work/07665/shrey911/ls6/LRP_eccov4r4_data'
mainDir_r5 = '/work/07665/shrey911/ls6/LRP_eccov4r5_data'
gridDir  = mainDir_r5 + '/GRID'
thetaDir = mainDir_r5 + '/SST_day_mean'
thetaDir_ext = mainDir_r5 + '/SST_day_mean_ext_2020_2023_Jun'

# For Sverdrup
# ds_r4 = xr.open_dataset(f'/scratch2/shreyas/LRP_eccov4r4_data/thetaSurfECCOv4r4.nc')
# For LS6
ds_r4 = xr.open_dataset(mainDir_r4 + '/thetaSurfECCOv4r4.nc')

# SSH has to be kept because someone used the SSH metadata for SST, 
# It's not a bug in this code but a hack to handle an existing bug.
temp = xmitgcm.open_mdsdataset(data_dir = thetaDir,
                             grid_dir = gridDir,
                             extra_variables = dict(SSH = dict(dims=['k','j','i'],
                                                               attrs = dict(standard_name="SST",
                                                                            long_name="Sea Surface Temperature",
                                                                            units="degC"))))
temp["SST"] = temp["SSH"]
temp = temp.drop(["SSH"])
    
temp_ext = xmitgcm.open_mdsdataset(data_dir = thetaDir_ext,
                                 grid_dir = gridDir,
                                 extra_variables = dict(SST = dict(dims=['k','j','i'],
                                                                   attrs = dict(standard_name="SST",
                                                                                long_name="Sea Surface Temperature",
                                                                                units="degC"))))

ds_r5 = xr.concat([temp, temp_ext], "time")

In [4]:
ds_SST = xr.open_dataset(mainDir_r5+'/SST_all.nc')
SST = ds_SST['SST'].data

In [5]:
hFacC = ecco.read_llc_to_tiles(gridDir, 'hFacC.data')
hFacC_mask = hFacC > 0
hFacC_mask = hFacC_mask.astype(float)

XC = ds_r4['XC'].data
YC = ds_r4['YC'].data

latMask = YC > -20.0
latMask = latMask.astype(float)

maskFinal = hFacC_mask * latMask
NaNmaskFinal = np.copy(maskFinal)
NaNmaskFinal[NaNmaskFinal == 0] = np.nan

da_XC = xr.DataArray(
    data=ds_r4['XC'].data,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="XC"),
)

da_YC = xr.DataArray(
    data=ds_r4['YC'].data,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="YC"),
)

da_hFacC_mask = xr.DataArray(
    data=hFacC_mask,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="hFacC mask 2D 1 if > 0, else 0"),
)

da_latMask = xr.DataArray(
    data=latMask,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="Latitude Mask 1 if > -20, else 0"),
)

da_maskFinal = xr.DataArray(
    data=maskFinal,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="Mask 2D 1 if > 0, else 0"),
)

da_NaNmaskFinal = xr.DataArray(
    data=NaNmaskFinal,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="Mask 2D True if > 0, else NaN"),
)

wetpoints = np.nonzero(maskFinal.data)
da_wetpoints = xr.DataArray(
    data=np.asarray(wetpoints),
    dims=["wetpoints_dim", "num_wetpoints"],
    coords=dict(
        wetpoints_dim = np.arange(np.asarray(wetpoints).shape[0], dtype = int),
        num_wetpoints = np.arange(np.asarray(wetpoints).shape[1], dtype = int),
    ),
    attrs=dict(description="indices of wetpoints in the order (tile, j, i) in the three rows"),
)

da_SST = xr.DataArray(
    data=SST,
    dims=["time", "tile", "j", "i"],
    coords=dict(
        time = ds_r5['time'].data[:SST.shape[0]],
        tile = ds_r4['tile'].data,
        j    = ds_r4['j'].data,
        i    = ds_r4['i'].data,
    ),
    attrs=dict(description="SST field in llc format"),
)

ds = xr.Dataset()
ds = ds.assign(XC           = da_XC,
               YC           = da_YC,
               hFacC_mask   = da_hFacC_mask,
               latMask      = da_latMask,
               maskFinal    = da_maskFinal,
               NaNmaskFinal = da_NaNmaskFinal,
               wetpoints    = da_wetpoints,
               SST          = da_SST)

load_binary_array: loading file /work/07665/shrey911/ls6/LRP_eccov4r5_data/GRID/hFacC.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4


In [6]:
def anomalize_new(field, num_years = 31, first_leap_year_idx = 0):
    
    leap_yr_offsets_jan_feb   = np.array([0,1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5,6,6,6,6,7,7,7,7,8,8])
    leap_yr_offsets_after_feb = np.array([1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5,6,6,6,6,7,7,7,7,8,8,8])

    if len(field.shape) > 1:
        seasonal_trend = np.zeros((366, field.shape[1]))
    else:
        seasonal_trend = np.zeros((366,))

    #### Calculate seasonal trend
    
    # Jan 1 - Feb 28
    for d in range(59):
        same_cal_days_idx=[d+365*year+leap_yr_offsets_jan_feb[year] for year in range(num_years)]
        # Remove mean
        field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                        axis=0, 
                                                        type='constant', 
                                                        overwrite_data=False)
        # Remove linear trend
        field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                        axis=0, 
                                                        type='linear', 
                                                        overwrite_data=False)
    
    # Feb 29 starting 1996, so year 2 in 0-indexing
    same_cal_days_idx=[365*year+59+int(year/4) for year in range(first_leap_year_idx,num_years,4)]
    # Remove mean
    field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                    axis=0, 
                                                    type='constant', 
                                                    overwrite_data=False)
    # Remove linear trend
    field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                    axis=0, 
                                                    type='linear', 
                                                    overwrite_data=False)
            
    # Mar 1 - Dec 31
    for d in range(60,366):
        same_cal_days_idx=[d-1+365*year+leap_yr_offsets_after_feb[year] for year in range(num_years)]
        # Remove mean
        field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                        axis=0, 
                                                        type='constant', 
                                                        overwrite_data=False)
        # Remove linear trend
        field[same_cal_days_idx] = scipy.signal.detrend(field[same_cal_days_idx], 
                                                        axis=0, 
                                                        type='linear', 
                                                        overwrite_data=False)

    return field

In [7]:
X = ds['SST'].data[:,wetpoints[0],wetpoints[1],wetpoints[2]].copy()
X = anomalize_new(X)
X_full = X.copy()
X = X[30:-30]

y = ds['SST'].isel(tile = 10, j = 1, i = 43).data.copy()
y = anomalize_new(y)
# https://stackoverflow.com/questions/13728392/moving-average-or-running-mean
y = np.convolve(y, np.ones(61)/61, mode='valid')
oneHotCost = np.zeros((y.shape[0], 2), dtype = int)
oneHotCost[:,0] = y >= 0.0
oneHotCost[:,1] = y <  0.0

In [8]:
da_X = xr.DataArray(
    data=X,
    dims=["time_allData", "num_wetpoints"],
    coords=dict(
        time_allData  = ds['time'].data[30:-30],
        num_wetpoints = ds['num_wetpoints'].data,
    ),
    attrs=dict(description="All data as matrix X; Mean removed and delinearized for each calendar day."),
)

da_y = xr.DataArray(
    data=y,
    dims=["time_allData"],
    coords=dict(
        time_allData  = ds['time'].data[30:-30],
    ),
    attrs=dict(description="All cost function y; Mean removed and delinearized for each calendar day."),
)

da_X_full = xr.DataArray(
    data=X_full,
    dims=["time", "num_wetpoints"],
    coords=dict(
        time          = ds['time'],
        num_wetpoints = ds['num_wetpoints'].data,
    ),
    attrs=dict(description="All data without accounting for conv filter as matrix X_full; Mean removed and delinearized for each calendar day."),
)

da_oneHotCost = xr.DataArray(
    data=oneHotCost,
    dims=["time_allData", "NN_output_dim"],
    coords=dict(
        time_allData  = ds['time'].data[30:-30],
        NN_output_dim = np.array([0,1]),
    ),
    attrs=dict(description="All cost function as one-hot vector."),
)

ds = ds.assign(X          = da_X,
               y          = da_y,
               X_full     = da_X_full,
               oneHotCost = da_oneHotCost)

## Classification

In [9]:
def LRP(model, model_wo_softmax, X, y_true, lagSteps, lrp_methods, 
        suffix, normalizeDict, **kwargs):

    result = {}
    cost_NN = model.predict(X)
    result[f'score_{suffix}'] = model.evaluate(X, y_true, verbose=0)

    pred_NN = cost_NN.copy()
    pred_NN[:,0] = pred_NN[:,0] > 0.5
    pred_NN[:,1] = pred_NN[:,1] > 0.5

    idx_NN_pos = []
    idx_NN_neg = []

    if lagSteps >= 0:
        for i in range(len(y_true[lagSteps:,0])):
            if y_true[lagSteps+i,0] == 1 and pred_NN[i,0] == 1:
                idx_NN_pos.append(i)
            if y_true[lagSteps+i,1] == 1 and pred_NN[i,1] == 1:
                idx_NN_neg.append(i)
    else:
        for i in range(len(y_true[:lagSteps,0])):
            if y_true[i,0] == 1 and pred_NN[i-lagSteps,0] == 1:
                idx_NN_pos.append(i-lagSteps)
            if y_true[i,1] == 1 and pred_NN[i-lagSteps,1] == 1:
                idx_NN_neg.append(i-lagSteps)

    result[f'idx_NN_pos'] = idx_NN_pos
    result[f'idx_NN_neg'] = idx_NN_neg

    rel = np.zeros((len(idx_NN_pos), 13, 90, 90))
    rel[:,:,:,:] = np.nan
    rel[:,wetpoints[0],wetpoints[1],wetpoints[2]] = X[idx_NN_pos]
    result[f'samples_correct_pos_{suffix}'] = rel

    rel = np.zeros((len(idx_NN_neg), 13, 90, 90))
    rel[:,:,:,:] = np.nan
    rel[:,wetpoints[0],wetpoints[1],wetpoints[2]] = X[idx_NN_neg]
    result[f'samples_correct_neg_{suffix}'] = rel
    
    for method in lrp_methods:

        title = method['title']
        
        print(f'Analyze using {title} for {suffix} data')
        
        Xplain = XAIR(model_wo_softmax, method, 'classic', X[idx_NN_pos[:10]], 
                      normalizeDict, **kwargs)
        a, _  = Xplain.quick_analyze()
        
        rel = np.zeros((a.shape[0], 13, 90, 90))
        rel[:,:,:,:] = np.nan
        rel[:,wetpoints[0],wetpoints[1],wetpoints[2]] = a
        result[method['title']+f'_pos_{suffix}'] = rel

        Xplain = XAIR(model_wo_softmax, method, 'classic', X[idx_NN_neg[:10]], 
                      normalizeDict, **kwargs)
        a, _  = Xplain.quick_analyze()
        
        rel = np.zeros((a.shape[0], 13, 90, 90))
        rel[:,:,:,:] = np.nan
        rel[:,wetpoints[0],wetpoints[1],wetpoints[2]] = a
        result[method['title']+f'_neg_{suffix}'] = rel
        
    return result

def quickSetup(X, y,
               test_split_frac,
               val_split_frac,
               lrp_methods,
               lagSteps,
               decay_func = None,
               **NNkwargs):
    
    result = {}

    idx = int(X.shape[0]*(1-test_split_frac))
    X_train = X[:idx]
    oneHotCost_train = oneHotCost[:idx]
    X_test = X[idx:]
    oneHotCost_test = oneHotCost[idx:]

    keras.backend.clear_session()
    sgd = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    NNkwargs['optimizer'] = sgd

    # Split the data into train and validation sets
    if lagSteps > 0:
        x_t, x_v, oneHotCost_t, oneHotCost_v = train_test_split(X_train[:-lagSteps], oneHotCost_train[lagSteps:], 
                                                                test_size=val_split_frac, shuffle= True, random_state=42)
    elif lagSteps == 0:
        x_t, x_v, oneHotCost_t, oneHotCost_v = train_test_split(X_train, oneHotCost_train, 
                                                                test_size=val_split_frac, shuffle= True, random_state=42)
    else:
        x_t, x_v, oneHotCost_t, oneHotCost_v = train_test_split(X_train[-lagSteps:], oneHotCost_train[:lagSteps], 
                                                                test_size=val_split_frac, shuffle= True, random_state=42)

    K = TrainFullyConnectedNN(x_t, oneHotCost_t, validation_data = (x_v, oneHotCost_v), **NNkwargs)
                   
    best_model = K.quickTrain(decay_func)
    best_model_wo_softmax = innvestigate.model_wo_softmax(best_model)
                   
    result['QoI_predict'] = best_model.predict(X)
    result['QoI_predict_train'] = best_model.predict(X_train)
    result['QoI_predict_test'] = best_model.predict(X_test)
                   
    normalizeDict = {'bool_': True, 'kind': 'MaxAbs'}
    kwargs = {'y_ref': 0.00}

    # LRP for all data
    result_train = LRP(best_model, best_model_wo_softmax, 
                       X, oneHotCost, lagSteps, lrp_methods, 
                       suffix = 'all', normalizeDict = normalizeDict, **kwargs)
    result.update(result_train)
    
    # # LRP for training data
    # result_train = LRP(best_model, best_model_wo_softmax, 
    #                    X_train, oneHotCost_train, lagSteps, lrp_methods, 
    #                    suffix = 'train', normalizeDict = normalizeDict, **kwargs)
    # result.update(result_train)

    # # LRP for test data
    # result_test = LRP(best_model, best_model_wo_softmax, 
    #                    X_test, oneHotCost_test, lagSteps, lrp_methods, 
    #                    suffix = 'test', normalizeDict = normalizeDict, **kwargs)
    # result.update(result_test)
        
    return result

In [10]:
Layers = [{'size': X.shape[1], 'activation': None     , 'use_bias': None},
          {'size': 8         , 'activation': 'relu'   , 'use_bias': True, 
           'l1_w_reg': 0.0, 'l1_b_reg': 0.0, 'l2_w_reg': 10.0, 'l2_b_reg': 10.0},
          {'size': 8         , 'activation': 'relu'   , 'use_bias': True, 
           'l1_w_reg': 0.0, 'l1_b_reg': 0.0, 'l2_w_reg': 0.01, 'l2_b_reg': 0.01},
          {'size': 2         , 'activation': 'softmax', 'use_bias': True, 
           'l1_w_reg': 0.0, 'l1_b_reg': 0.0, 'l2_w_reg': 0.01, 'l2_b_reg': 0.01,
           'bias_constraint': NonNeg()}]

Losses = [{'kind': 'categorical_crossentropy', 'weight': 1.0}]

LRPDict_theta = {}
lagStepsList = [-60,-30,0]#,30,60,90,120,150,180]

# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.01
    drop = 0.70710678118 # 0.5^0.5
    epochs_drop = 50
    lrate = initial_lrate * drop**np.floor((1+epoch)/epochs_drop)
    return lrate

methods = [# dict(name='lrp.alpha_1_beta_0', title = 'LRP-A1B0', optParams = {}),
           # dict(name='lrp.alpha_1_beta_0', title = 'LRP-A1B0-W2', optParams = {'input_layer_rule':'WSquare'}),
           dict(name='lrp.alpha_1_beta_0', title = 'LRP-A1B0-B' , optParams = {'input_layer_rule':'Bounded'})]

In [11]:
weight_for_pos = len(oneHotCost[:-2161,0]) / np.sum(oneHotCost[:-2161,0])
weight_for_neg = len(oneHotCost[:-2161,1]) / np.sum(oneHotCost[:-2161,1])
class_weight = {0: weight_for_pos, 1: weight_for_neg}
class_weight

{0: 2.18692936088419, 1: 1.84251012145749}

In [12]:
for i in range(len(lagStepsList)):

    print(f'Lag: {lagStepsList[i]} days, for Theta')
    
    NNkwargs = {'losses': Losses,
                'metrics': [metricF1, # Custom f1 since metrics.F1Score(name='f1') is not available before tf v2.13
                            'accuracy',
                            'categorical_crossentropy',
                            # metrics.CategoricalCrossentropy(name='cross_entropy'),  # (model's loss-L2 reg loss)
                            # metrics.MeanSquaredError(name='brier_score'),
                            # metrics.TruePositives(name='tp'),
                            # metrics.FalsePositives(name='fp'),
                            # metrics.TrueNegatives(name='tn'),
                            # metrics.FalseNegatives(name='fn'), 
                            # metrics.BinaryAccuracy(name='accuracy'),
                            # metrics.Precision(name='precision'),
                            # metrics.Recall(name='recall'),
                            # metrics.AUC(name='auc'),
                            # metrics.AUC(name='prc', curve='PR'), # precision-recall curve],
                           ],
                'batch_size': 128, 'epochs': 1, #'validation_split': 0.1,
                'filename': f'model{lagStepsList[i]}_noL1', 'dirname': os.path.abspath(''),
                'random_nn_seed': 42, 'class_weight': class_weight,
                'custom_objects': {'metricF1': metricF1}, 'verbose': 2}
    
    LRPDict_theta[f'LRP{lagStepsList[i]}'] = quickSetup(X, y, test_split_frac = 2161.0/11263.0,
                                                        val_split_frac = 0.2,
                                                        lrp_methods = methods,
                                                        lagSteps = lagStepsList[i],
                                                        decay_func = step_decay,
                                                        layers = Layers, **NNkwargs)

Lag: -60 days, for Theta
Train on 7233 samples, validate on 1809 samples


2024-04-17 21:27:01.057854: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38221 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:21:00.0, compute capability: 8.0
2024-04-17 21:27:01.059857: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38221 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0
2024-04-17 21:27:01.061883: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 38221 MB memory:  -> device: 2, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:e2:00.0, compute capability: 8.0
2024-04-17 21:27:01.080484: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2024-04-17 21:27:05.123275: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] Ten


Epoch 1: val_loss improved from inf to 3.09536, saving model to /home1/07665/shrey911/XAIRT/examples_TomsQoI/model-60_noL1.h5
7233/7233 - 6s - loss: 9.6708 - metricF1: 0.7140 - accuracy: 0.7135 - categorical_crossentropy: 0.8069 - val_loss: 3.0954 - val_metricF1: 0.6504 - val_accuracy: 0.6506 - val_categorical_crossentropy: 1.0621 - lr: 0.0100 - 6s/epoch - 767us/sample
Analyze using LRP-A1B0-B for all data
Lag: -30 days, for Theta
Train on 7257 samples, validate on 1815 samples


2024-04-17 21:27:13.755403: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38221 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:21:00.0, compute capability: 8.0
2024-04-17 21:27:13.757401: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38221 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0
2024-04-17 21:27:13.759412: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 38221 MB memory:  -> device: 2, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:e2:00.0, compute capability: 8.0



Epoch 1: val_loss improved from inf to 0.93339, saving model to /home1/07665/shrey911/XAIRT/examples_TomsQoI/model-30_noL1.h5
7257/7257 - 1s - loss: 9.9504 - metricF1: 0.7051 - accuracy: 0.7054 - categorical_crossentropy: 0.9220 - val_loss: 0.9334 - val_metricF1: 0.7482 - val_accuracy: 0.7488 - val_categorical_crossentropy: 0.6112 - lr: 0.0100 - 824ms/epoch - 114us/sample
Analyze using LRP-A1B0-B for all data
Lag: 0 days, for Theta
Train on 7281 samples, validate on 1821 samples


2024-04-17 21:27:21.576081: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38221 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:21:00.0, compute capability: 8.0
2024-04-17 21:27:21.578083: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38221 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0
2024-04-17 21:27:21.580098: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 38221 MB memory:  -> device: 2, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:e2:00.0, compute capability: 8.0



Epoch 1: val_loss improved from inf to 1.20418, saving model to /home1/07665/shrey911/XAIRT/examples_TomsQoI/model0_noL1.h5
7281/7281 - 1s - loss: 9.8030 - metricF1: 0.7308 - accuracy: 0.7309 - categorical_crossentropy: 0.8720 - val_loss: 1.2042 - val_metricF1: 0.7455 - val_accuracy: 0.7392 - val_categorical_crossentropy: 0.5077 - lr: 0.0100 - 802ms/epoch - 110us/sample
Analyze using LRP-A1B0-B for all data


In [13]:
import innvestigate
import innvestigate.utils as iutils
from innvestigate.analyzer.base import AnalyzerBase

model = keras.models.load_model('model0_noL1.h5', custom_objects={'metricF1': metricF1})
model_wo_softmax = innvestigate.model_wo_softmax(model)
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax, input_layer_rule='Bounded')
sample = LRPDict_theta['LRP0']['samples_correct_neg_all'][0,wetpoints[0],wetpoints[1],wetpoints[2]][np.newaxis,:]

a = Analyze.analyze(sample)

In [14]:
model_wo_softmax.predict(LRPDict_theta['LRP0']['samples_correct_neg_all'][:,wetpoints[0],wetpoints[1],wetpoints[2]])

array([[-0.36320633, -0.21332543],
       [-0.43081477, -0.1625068 ],
       [-0.50695264, -0.11905544],
       ...,
       [-0.13323672,  0.53053445],
       [-0.12375152,  0.5035467 ],
       [-0.12355828,  0.49237347]], dtype=float32)

In [15]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax, input_layer_rule='Bounded')
sample = LRPDict_theta['LRP0']['samples_correct_neg_all'][0,wetpoints[0],wetpoints[1],wetpoints[2]][np.newaxis,:]

a = Analyze.analyze(sample)
np.sum(a)

-0.0727483

In [16]:
Analyze = innvestigate.create_analyzer('lrp.z', model_wo_softmax)
sample = LRPDict_theta['LRP0']['samples_correct_neg_all'][0,wetpoints[0],wetpoints[1],wetpoints[2]][np.newaxis,:]

a = Analyze.analyze(sample)
np.sum(a)

-0.5121174

In [17]:
model_wo_softmax.predict(LRPDict_theta['LRP0']['samples_correct_pos_all'][:,wetpoints[0],wetpoints[1],wetpoints[2]])

array([[ 0.8836373 , -0.6698701 ],
       [ 0.91328305, -0.6360027 ],
       [ 0.91239804, -0.64911586],
       ...,
       [ 0.30899364,  0.11315773],
       [ 0.28144538,  0.16191523],
       [ 0.28097343,  0.23456167]], dtype=float32)

In [18]:
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax, input_layer_rule='Bounded')
sample = LRPDict_theta['LRP0']['samples_correct_pos_all'][0,wetpoints[0],wetpoints[1],wetpoints[2]][np.newaxis,:]

a = Analyze.analyze(sample)
np.sum(a)

1.015388

In [19]:
Analyze = innvestigate.create_analyzer('lrp.z', model_wo_softmax)
sample = LRPDict_theta['LRP0']['samples_correct_pos_all'][0,wetpoints[0],wetpoints[1],wetpoints[2]][np.newaxis,:]

a = Analyze.analyze(sample)
np.sum(a)

1.2893748

In [20]:
model_wo_softmax.layers[-1].get_weights()[0], model_wo_softmax.layers[-1].get_weights()[1]

(array([[-0.13878407, -0.07667916],
        [-0.3684209 ,  0.03289204],
        [-0.04403416, -0.19840208],
        [ 0.17202522, -0.3319991 ],
        [ 0.1678728 , -0.3053772 ],
        [ 0.35115594,  0.72690094],
        [ 0.6358902 ,  0.23027413],
        [ 0.12554084,  0.1512276 ]], dtype=float32),
 array([-0.        ,  0.10137586], dtype=float32))

In [21]:
model_wo_softmax.layers[-2].get_weights()[0], model_wo_softmax.layers[-2].get_weights()[1]

(array([[-0.15695263, -0.1890395 , -0.38048807, -0.19095354, -0.29115474,
         -0.01246644, -0.51051366,  0.23851246],
        [ 0.16469038, -0.35451564,  0.24644446,  0.28855434,  0.85127294,
          0.11622895, -0.6371932 ,  0.4977694 ],
        [-0.0668036 ,  0.5177149 , -0.09532668, -0.29096928, -0.0727115 ,
         -0.32019994, -0.2376931 , -0.4644658 ],
        [-0.03370003,  1.0580934 , -0.33147657,  0.37784266,  0.05481944,
          0.42534202,  0.0287399 ,  0.10487936],
        [-0.09779887,  0.05178871,  0.3458608 , -0.43983147,  0.5959899 ,
         -0.28019264, -0.03330053, -0.7696547 ],
        [-0.28288943, -0.3029981 ,  0.20015028,  0.3683088 , -0.41913405,
         -0.08374354,  0.79560924, -0.36694312],
        [ 0.08402578, -0.07843378,  0.17449471, -0.58836085,  0.37387958,
          0.1981437 , -0.3537311 , -0.14975516],
        [ 0.32105803, -0.19068165, -0.39748433,  0.31145713,  0.12014196,
         -0.41334882, -0.4318624 , -0.24736986]], dtype=float32),

In [22]:
import innvestigate
import innvestigate.utils as iutils
from innvestigate.analyzer.base import AnalyzerBase

model = keras.models.load_model('model-30_noL1.h5', custom_objects={'metricF1': metricF1})
model_wo_softmax = innvestigate.model_wo_softmax(model)
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax, input_layer_rule='Bounded')
sample = LRPDict_theta['LRP0']['samples_correct_neg_all'][0,wetpoints[0],wetpoints[1],wetpoints[2]][np.newaxis,:]

a = Analyze.analyze(sample)

In [23]:
model_wo_softmax.predict(LRPDict_theta['LRP0']['samples_correct_neg_all'][:,wetpoints[0],wetpoints[1],wetpoints[2]])

array([[ 0.07533138,  0.13981977],
       [ 0.07316314,  0.13468629],
       [ 0.07221177,  0.13720539],
       ...,
       [-0.08523329, -0.01531784],
       [-0.08992133, -0.02229644],
       [-0.09892866, -0.03854377]], dtype=float32)

In [24]:
model_wo_softmax.layers[-1].get_weights()[0], model_wo_softmax.layers[-1].get_weights()[1]

(array([[-0.10096561, -0.11449764],
        [-0.07321362, -0.2623151 ],
        [-0.2345912 , -0.00784506],
        [-0.08942901, -0.07054525],
        [-0.00458508, -0.13291907],
        [ 0.42924574,  0.64881086],
        [ 0.5474082 ,  0.3187558 ],
        [ 0.1481839 ,  0.12858473]], dtype=float32),
 array([0.00531946, 0.09743632], dtype=float32))

In [25]:
import innvestigate
import innvestigate.utils as iutils
from innvestigate.analyzer.base import AnalyzerBase

model = keras.models.load_model('model-60_noL1.h5', custom_objects={'metricF1': metricF1})
model_wo_softmax = innvestigate.model_wo_softmax(model)
Analyze = innvestigate.create_analyzer('lrp.alpha_1_beta_0', model_wo_softmax, input_layer_rule='Bounded')
sample = LRPDict_theta['LRP0']['samples_correct_neg_all'][0,wetpoints[0],wetpoints[1],wetpoints[2]][np.newaxis,:]

a = Analyze.analyze(sample)

In [26]:
model_wo_softmax.predict(LRPDict_theta['LRP0']['samples_correct_neg_all'][:,wetpoints[0],wetpoints[1],wetpoints[2]])

array([[ 0.5828423 , -1.26056   ],
       [ 0.5975521 , -1.3044808 ],
       [ 0.6133962 , -1.3482869 ],
       ...,
       [-2.7397432 ,  0.10793108],
       [-2.795439  ,  0.10581087],
       [-2.9008088 ,  0.10179965]], dtype=float32)

In [27]:
model_wo_softmax.layers[-1].get_weights()[0], model_wo_softmax.layers[-1].get_weights()[1]

(array([[-5.4818206e-04, -2.1491505e-01],
        [-3.2322422e-01, -1.2304485e-02],
        [-1.3072477e-01, -1.1171138e-01],
        [ 6.9510862e-02, -2.2948444e-01],
        [ 1.0696639e-01, -2.4447083e-01],
        [ 4.1766283e-01,  6.6039354e-01],
        [ 5.7215530e-01,  2.9400894e-01],
        [ 7.0390165e-02,  2.0637855e-01]], dtype=float32),
 array([0.00093038, 0.21226293], dtype=float32))